In [3]:
import pandas as pd

### TPOT

In [4]:
drug_data_for_tpot = pd.read_csv('/home/kdt-admin/miniconda3/envs/JS/WASSUP_Project_Team7_2-1/data/drug_data_for_tpot.csv')

In [4]:
drug_data_for_tpot.head()

,Padding_re_con,encoded_drugName
0,"[7, 10, 4, 1, 494, 2198, 2492, 1197, 312, 3649...",3197
1,"[493, 2555, 1336, 19, 2455, 303, 614, 257, 48,...",1428
2,"[17, 1, 172, 885, 370, 359, 159, 190, 20, 1457...",1851
3,"[15, 13, 17, 475, 14, 12, 469, 50, 278, 15, 35...",2279
4,"[746, 134, 376, 37, 137, 11, 1808, 888, 338, 1...",515


In [6]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

X_tfidf = tfidf_vectorizer.fit_transform(drug_data_for_tpot['Padding_re_con'])

X = X_tfidf
y = drug_data_for_tpot['encoded_drugName']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tpot = TPOTClassifier(config_dict='TPOT sparse', generations=5, population_size=20, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)

print("Test accuracy:", tpot.score(X_test, y_test))

/home/kdt-admin/miniconda3/envs/JS/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")



Generation 1 - Current best internal CV score: 0.23301154321908113

Generation 2 - Current best internal CV score: 0.23301154321908113

Generation 3 - Current best internal CV score: 0.23301154321908113

Generation 4 - Current best internal CV score: 0.23301154321908113

Generation 5 - Current best internal CV score: 0.2859573573539388

Best pipeline: MultinomialNB(SelectPercentile(input_matrix, percentile=16), alpha=0.001, fit_prior=True)
Test accuracy: 0.28333281303655605

In [ ]:
tpot.export('drugdata_tpot_pipeline.py')

In [35]:
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

tfidf_vectorizer = TfidfVectorizer()

X_tfidf = tfidf_vectorizer.fit_transform(drug_data_for_tpot['Padding_re_con'])

X = X_tfidf
y = drug_data_for_tpot['encoded_drugName']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
tpot_pipeline = make_pipeline(
    SelectPercentile(score_func=f_classif, percentile=16),
    MultinomialNB(alpha=0.001, fit_prior=True)
)

tpot_pipeline.fit(X_train, y_train)

accuracy = tpot_pipeline.score(X_test, y_test)

/home/kdt-admin/miniconda3/envs/JS/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/kdt-admin/miniconda3/envs/JS/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/home/kdt-admin/miniconda3/envs/JS/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [30]:
y_pred = tpot_pipeline.predict(X_test)

accuracy = (y_pred == y_test).mean()
print("Test accuracy:", accuracy)

Test accuracy: 0.28333281303655605


### 가중치 추가

In [37]:
drug_data_pp_weight= pd.read_csv('/home/kdt-admin/miniconda3/envs/JS/WASSUP_Project_Team7_2-1/data/drug_data_pp_weight.csv')

In [38]:
drug_data_pp_weight.head()

,Padding_re_con,encoded_drugName,sentiment,uC_rank
0,"[7, 10, 4, 1, 494, 2198, 2492, 1197, 312, 3649...",3197,3,2
1,"[493, 2555, 1336, 19, 2455, 303, 614, 257, 48,...",1428,3,5
2,"[17, 1, 172, 885, 370, 359, 159, 190, 20, 1457...",1851,2,2
3,"[15, 13, 17, 475, 14, 12, 469, 50, 278, 15, 35...",2279,3,1
4,"[746, 134, 376, 37, 137, 11, 1808, 888, 338, 1...",515,3,4


In [40]:
drug_data_pp_weight.shape

(159958, 4)

#### 전체 drugName

In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import numpy as np

def apply_importance(df):
    importance = df['uC_rank']  + df['sentiment']
    return importance


preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), 'Padding_re_con')
    ],
    remainder='passthrough'
)

tpot_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('selector', SelectPercentile(score_func=f_classif, percentile=16)),  
    ('classifier', MultinomialNB(alpha=0.001, fit_prior=True))  
])

X = drug_data_pp_weight[['sentiment', 'uC_rank', 'Padding_re_con']]
y = drug_data_pp_weight['encoded_drugName']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 가중치 계산 및 추가
X_train['importance'] = X_train.apply(apply_importance, axis=1)
X_test['importance'] = X_test.apply(apply_importance, axis=1)

# 학습
tpot_pipeline.fit(X_train, y_train)

# 정확도
accuracy = tpot_pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

/home/kdt-admin/miniconda3/envs/JS/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


Accuracy: 0.28632158039509875


#### TOP 100 drugName

In [43]:
drug_data_drugTop100 = pd.read_csv('/home/kdt-admin/miniconda3/envs/JS/WASSUP_Project_Team7_2-1/data/drug_data_drugTop100.csv')
drug_data_drugTop100.head()

,Padding_re_con,encoded_drugName,sentiment,uC_rank
0,"[746, 134, 376, 37, 137, 11, 1808, 888, 338, 1...",515,3,4
1,"[895, 3019, 195, 35, 284, 416, 148, 35, 320, 3...",1745,1,1
2,"[807, 77, 37, 92, 356, 1797, 15, 9, 807, 433, ...",259,3,3
3,"[21, 133, 26, 32, 77, 1392, 2755, 4, 43, 24, 1...",1179,3,1
4,"[29, 76, 52, 19, 9, 6, 154, 422, 7, 72, 97, 16...",3056,3,2


In [48]:
X = drug_data_drugTop100[['sentiment', 'uC_rank', 'Padding_re_con']]
y = drug_data_drugTop100['encoded_drugName']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train['importance'] = X_train.apply(apply_importance, axis=1)
X_test['importance'] = X_test.apply(apply_importance, axis=1)

tpot_pipeline.fit(X_train, y_train)

accuracy = tpot_pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.40891897272546285


#### TOP 50 drugName

In [44]:
drug_data_drugTop50 = pd.read_csv('/home/kdt-admin/miniconda3/envs/JS/WASSUP_Project_Team7_2-1/data/drug_data_drugTop50.csv')
drug_data_drugTop50.head()

,Padding_re_con,encoded_drugName,sentiment,uC_rank
0,"[895, 3019, 195, 35, 284, 416, 148, 35, 320, 3...",1745,1,1
1,"[21, 133, 26, 32, 77, 1392, 2755, 4, 43, 24, 1...",1179,3,1
2,"[29, 76, 52, 19, 9, 6, 154, 422, 7, 72, 97, 16...",3056,3,2
3,"[9, 375, 66, 585, 449, 370, 1, 144, 1703, 1, 3...",2145,1,1
4,"[1, 1398, 39, 762, 51, 90, 18, 36, 251, 39, 53...",1784,3,2


In [49]:
X = drug_data_drugTop50[['sentiment', 'uC_rank', 'Padding_re_con']]
y = drug_data_drugTop50['encoded_drugName']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train['importance'] = X_train.apply(apply_importance, axis=1)
X_test['importance'] = X_test.apply(apply_importance, axis=1)

tpot_pipeline.fit(X_train, y_train)

accuracy = tpot_pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.49314416301656827


#### TOP 30 drugName

In [45]:
drug_data_drugTop30 = pd.read_csv('/home/kdt-admin/miniconda3/envs/JS/WASSUP_Project_Team7_2-1/data/drug_data_drugTop30.csv')
drug_data_drugTop30.head()

,Padding_re_con,encoded_drugName,sentiment,uC_rank
0,"[895, 3019, 195, 35, 284, 416, 148, 35, 320, 3...",1745,1,1
1,"[21, 133, 26, 32, 77, 1392, 2755, 4, 43, 24, 1...",1179,3,1
2,"[9, 375, 66, 585, 449, 370, 1, 144, 1703, 1, 3...",2145,1,1
3,"[375, 338, 238, 280, 106, 65, 20, 227, 190, 22...",1186,3,1
4,"[356, 27, 85, 163, 1, 115, 1237, 90, 3, 31, 53...",2798,3,1


In [50]:
X = drug_data_drugTop30[['sentiment', 'uC_rank', 'Padding_re_con']]
y = drug_data_drugTop30['encoded_drugName']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train['importance'] = X_train.apply(apply_importance, axis=1)
X_test['importance'] = X_test.apply(apply_importance, axis=1)

tpot_pipeline.fit(X_train, y_train)

accuracy = tpot_pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5245029758136001


#### TOP 10 drugName

In [46]:
drug_data_drugTop10 = pd.read_csv('/home/kdt-admin/miniconda3/envs/JS/WASSUP_Project_Team7_2-1/data/drug_data_drugTop10.csv')
drug_data_drugTop10.head()

,Padding_re_con,encoded_drugName,sentiment,uC_rank
0,"[895, 3019, 195, 35, 284, 416, 148, 35, 320, 3...",1745,1,1
1,"[21, 133, 26, 32, 77, 1392, 2755, 4, 43, 24, 1...",1179,3,1
2,"[9, 375, 66, 585, 449, 370, 1, 144, 1703, 1, 3...",2145,1,1
3,"[375, 338, 238, 280, 106, 65, 20, 227, 190, 22...",1186,3,1
4,"[356, 27, 85, 163, 1, 115, 1237, 90, 3, 31, 53...",2798,3,1


In [51]:
X = drug_data_drugTop10[['sentiment', 'uC_rank', 'Padding_re_con']]
y = drug_data_drugTop10['encoded_drugName']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train['importance'] = X_train.apply(apply_importance, axis=1)
X_test['importance'] = X_test.apply(apply_importance, axis=1)

tpot_pipeline.fit(X_train, y_train)

accuracy = tpot_pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.6802850356294536


#### TOP 3 drugName

In [52]:
drug_data_drugTop3 = pd.read_csv('/home/kdt-admin/miniconda3/envs/JS/WASSUP_Project_Team7_2-1/data/drug_data_drugTop3.csv')
drug_data_drugTop3.head()

,Padding_re_con,encoded_drugName,sentiment,uC_rank
0,"[895, 3019, 195, 35, 284, 416, 148, 35, 320, 3...",1745,1,1
1,"[375, 338, 238, 280, 106, 65, 20, 227, 190, 22...",1186,3,1
2,"[96, 61, 2, 320, 331, 152, 102, 44, 70, 8, 20,...",1186,3,1
3,"[345, 74, 76, 26, 66, 135, 2315, 136, 7, 63, 2...",1745,1,1
4,"[13, 568, 263, 2315, 1249, 568, 891, 39, 441, ...",1745,3,1


In [53]:
X = drug_data_drugTop3[['sentiment', 'uC_rank', 'Padding_re_con']]
y = drug_data_drugTop3['encoded_drugName']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train['importance'] = X_train.apply(apply_importance, axis=1)
X_test['importance'] = X_test.apply(apply_importance, axis=1)

tpot_pipeline.fit(X_train, y_train)

accuracy = tpot_pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9043927648578811
